In [7]:
import pymongo
from pymongo.mongo_client import MongoClient


uri = ""

# Create a new client and connect to the server
client = MongoClient(uri)
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)
    


Pinged your deployment. You successfully connected to MongoDB!


In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim import corpora, models
import string

# Descargar recursos necesarios de NLTK (ejecutar solo una vez)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Definir los conceptos
conceptos = [
    'Presión de los líderes del equipo',
    'Adaptación al equipo y buen compañerismo',
    'Tensión en el equipo de trabajo'
]

conceptos = ['Análisis del capacity actual y propuestas a clientes',
 'retrospectiva',
 'Opciones de capacitación interna',
 'Seguimiento en el capacity y capacitación a freelancers',
 'Falta de feedback']

# Tokenización y preprocesamiento
def preprocess(text):
    # Tokenizar
    tokens = word_tokenize(text.lower())
    # Quitar stopwords y puntuación
    stop_words = set(stopwords.words('spanish') + list(string.punctuation))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Lematización
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    return lemmatized_tokens

# Preprocesar los conceptos
preprocessed_concepts = [preprocess(concepto) for concepto in conceptos]

# Construir el diccionario y el corpus
dictionary = corpora.Dictionary(preprocessed_concepts)
corpus = [dictionary.doc2bow(text) for text in preprocessed_concepts]

# Entrenar el modelo LDA
lda_model = models.LdaModel(corpus, num_topics=2, id2word=dictionary, passes=15)

# Mostrar los temas
print(lda_model.print_topics())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\germa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[(0, '0.156*"falta" + 0.156*"feedback" + 0.155*"retrospectiva" + 0.054*"opciones" + 0.054*"interna" + 0.054*"capacitación" + 0.053*"freelancer" + 0.053*"capacity" + 0.053*"seguimiento" + 0.053*"análisis"'), (1, '0.135*"capacity" + 0.135*"capacitación" + 0.081*"actual" + 0.081*"clientes" + 0.081*"propuestas" + 0.081*"análisis" + 0.081*"seguimiento" + 0.081*"freelancer" + 0.081*"interna" + 0.081*"opciones"')]


In [2]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Crear modelo de Sentence Transformers
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

def agrupar_topicos(topicos, umbral=0.5):
    grupos = []
    
    for topico in topicos:
        # Obtener las incrustaciones de las oraciones
        embeddings = model.encode([topico])[0]
        
        # Determinar si el tópico pertenece a un grupo existente
        agregado = False
        for grupo in grupos:
            for t in grupo:
                similarity_score = cosine_similarity([embeddings], [model.encode([t])[0]])[0][0]
                if similarity_score > umbral:
                    grupo.add(topico)
                    agregado = True
                    break
            if agregado:
                break
        
        # Si el tópico no se agrupó, crear un nuevo grupo
        if not agregado:
            grupos.append({topico})
    
    return grupos

# Lista de tópicos
topicos = [
    'Adaptación al equipo y buen compañerismo',
    'Falta de tiempo para testear implementaciones',
    'Presión de los líderes del equipo',
    'Comidas poco llamativas en la oficina',
    'Espacios de trabajo no confortables',
    'Los espacios de trabajo no son suficientemente confortables',
    'Traumas por corregir errores y falta de recursos para pagar psicólogo'
]

topicos = ['Análisis del capacity actual y propuestas a clientes',
 'retrospectiva',
 'Opciones de capacitación interna',
 'Seguimiento en el capacity y capacitación a freelancers',
 'Falta de feedback']

grupos = agrupar_topicos(topicos)

for i, grupo in enumerate(grupos):
    print("Grupo", i+1, ":", grupo)

Grupo 1 : {'Seguimiento en el capacity y capacitación a freelancers', 'Análisis del capacity actual y propuestas a clientes'}
Grupo 2 : {'retrospectiva'}
Grupo 3 : {'Opciones de capacitación interna'}
Grupo 4 : {'Falta de feedback'}


In [34]:
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util

# Crear modelo de Sentence Transformers
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

def encontrar_concepto_medio(topicos, modelo):
    embeddings = [modelo.encode([topico])[0] for topico in topicos]
    cov_matrix = np.cov(np.array(embeddings).T)
    U, S, V = np.linalg.svd(cov_matrix)
    direccion_promedio = V[0]
    direccion_promedio = direccion_promedio / np.linalg.norm(direccion_promedio)
    concepto_medio = np.mean(embeddings, axis=0) + direccion_promedio
    return concepto_medio

def encontrar_tópico_descriptivo(vector, palabras, modelo):
    vector = torch.tensor(vector, dtype=torch.float32)
    embeddings_palabras = torch.tensor(modelo.encode(palabras), dtype=torch.float32)
    vecino_cercano = util.semantic_search(vector.reshape(1, -1), embeddings_palabras, top_k=1)[0][0]
    tópico_descriptivo = palabras[vecino_cercano['corpus_id']]
    return tópico_descriptivo

def agrupar_topicos(topicos, umbral=0.5):
    grupos = []
    
    for topico in topicos:
        embeddings_1 = model.encode([topico])[0]
        agregado = False
        for grupo in grupos:
            for t in grupo:
                embeddings_2 = model.encode([t])[0]
                similarity_score = cosine_similarity([embeddings_1], [embeddings_2])[0][0]
                if similarity_score > umbral:
                    grupo.add(topico)
                    agregado = True
                    break
            if agregado:
                break
        if not agregado:
            grupos.append({topico})
    
    return grupos

def encontrar_tópico_generalizador(topicos, modelo):
    embeddings = [modelo.encode([topico])[0] for topico in topicos]
    mean_embedding = np.mean(embeddings, axis=0)
    similitudes = [cosine_similarity([mean_embedding], [embedding])[0][0] for embedding in embeddings]
    index_tópico_generalizador = np.argmax(similitudes)
    tópico_generalizador = list(topicos)[index_tópico_generalizador]
    return tópico_generalizador

# Lista de tópicos
topicos = [
    'Adaptación al equipo y buen compañerismo',
    'Falta de tiempo para testear implementaciones',
    'Presión de los líderes del equipo',
    'Comidas poco llamativas en la oficina',
    'Espacios de trabajo no confortables',
    'Los espacios de trabajo no son suficientemente confortables',
    'Traumas por corregir errores y falta de recursos para pagar psicólogo'
]

topicos = ['Análisis del capacity actual y propuestas a clientes',
 'retrospectiva',
 'Opciones de capacitación interna',
 'Seguimiento en el capacity y capacitación a freelancers',
 'Falta de feedback']


# Calcular el concepto medio entre los tópicos
concepto_medio = encontrar_concepto_medio(topicos, model)

# Palabras relevantes para describir los tópicos
palabras = ['equipo', 'compañerismo', 'presión', 'trabajo', 'estrés', 'comida', 'oficina', 'espacios', 'confort', 'psicólogo']

# Encontrar el tópico descriptivo del grupo
tópico_descriptivo = encontrar_tópico_descriptivo(concepto_medio, palabras, model)
print("Tópico descriptivo del grupo:", tópico_descriptivo)

# Agrupar los tópicos
grupos = agrupar_topicos(topicos)

# Encontrar el tópico generalizador de cada grupo
for i, grupo in enumerate(grupos):
    print("\nGrupo", i+1, ":", grupo)
    tópico_generalizador = encontrar_tópico_generalizador(grupo, model)
    print("Tópico generalizador del grupo:", tópico_generalizador)


Tópico descriptivo del grupo: confort

Grupo 1 : {'Análisis del capacity actual y propuestas a clientes', 'Seguimiento en el capacity y capacitación a freelancers'}
Tópico generalizador del grupo: Análisis del capacity actual y propuestas a clientes

Grupo 2 : {'retrospectiva'}
Tópico generalizador del grupo: retrospectiva

Grupo 3 : {'Opciones de capacitación interna'}
Tópico generalizador del grupo: Opciones de capacitación interna

Grupo 4 : {'Falta de feedback'}
Tópico generalizador del grupo: Falta de feedback


In [1]:
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

# Crear modelo de Sentence Transformers
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

def encontrar_concepto_medio(topicos, modelo):
    embeddings = [modelo.encode([topico])[0] for topico in topicos]
    cov_matrix = np.cov(np.array(embeddings).T)
    U, S, V = np.linalg.svd(cov_matrix)
    direccion_promedio = V[0]
    direccion_promedio = direccion_promedio / np.linalg.norm(direccion_promedio)
    concepto_medio = np.mean(embeddings, axis=0) + direccion_promedio
    return concepto_medio

def encontrar_tópico_descriptivo(vector, palabras, modelo):
    vector = torch.tensor(vector, dtype=torch.float32)
    embeddings_palabras = torch.tensor(modelo.encode(palabras), dtype=torch.float32)
    vecino_cercano = util.semantic_search(vector.reshape(1, -1), embeddings_palabras, top_k=1)[0][0]
    tópico_descriptivo = palabras[vecino_cercano['corpus_id']]
    return tópico_descriptivo

def agrupar_topicos(topicos, umbral=0.5):
    grupos = []
    
    for topico in topicos:
        embeddings_1 = model.encode([topico])[0]
        agregado = False
        for grupo in grupos:
            for t in grupo:
                embeddings_2 = model.encode([t])[0]
                similarity_score = cosine_similarity([embeddings_1], [embeddings_2])[0][0]
                if similarity_score > umbral:
                    grupo.add(topico)
                    agregado = True
                    break
            if agregado:
                break
        if not agregado:
            grupos.append({topico})
    
    return grupos

def encontrar_tópico_generalizador(topicos, modelo):
    embeddings = [modelo.encode([topico])[0] for topico in topicos]
    mean_embedding = np.mean(embeddings, axis=0)
    similitudes = [cosine_similarity([mean_embedding], [embedding])[0][0] for embedding in embeddings]
    index_tópico_generalizador = np.argmax(similitudes)
    tópico_generalizador = list(topicos)[index_tópico_generalizador]
    return tópico_generalizador

# Lista de tópicos
topicos = [
    'Adaptación al equipo y buen compañerismo',
    'Falta de tiempo para testear implementaciones',
    'Presión de los líderes del equipo',
    'Comidas poco llamativas en la oficina',
    'Espacios de trabajo no confortables',
    'Los espacios de trabajo no son suficientemente confortables',
    'Traumas por corregir errores y falta de recursos para pagar psicólogo'
]

topicos = ['Análisis del capacity actual y propuestas a clientes',
 'retrospectiva',
 'Opciones de capacitación interna',
 'Seguimiento en el capacity y capacitación a freelancers',
 'Falta de feedback']

# Calcular el concepto medio entre los tópicos
concepto_medio = encontrar_concepto_medio(topicos, model)

# Agrupar los tópicos
grupos = agrupar_topicos(topicos)

# Encontrar el tópico generalizador de cada grupo
for i, grupo in enumerate(grupos):
    print("\nGrupo", i+1, ":", grupo)
    
    # Palabras relevantes para describir los tópicos del grupo
    palabras_grupo = ' '.join(grupo).split()
    
    # Encontrar el tópico descriptivo del grupo
    tópico_descriptivo = encontrar_tópico_descriptivo(concepto_medio, palabras_grupo, model)
    print("Tópico descriptivo del grupo:", tópico_descriptivo)

    # Encontrar el tópico generalizador del grupo
    tópico_generalizador = encontrar_tópico_generalizador(grupo, model)
    print("Tópico generalizador del grupo:", tópico_generalizador)



Grupo 1 : {'Análisis del capacity actual y propuestas a clientes', 'Seguimiento en el capacity y capacitación a freelancers'}
Tópico descriptivo del grupo: capacitación
Tópico generalizador del grupo: Análisis del capacity actual y propuestas a clientes

Grupo 2 : {'retrospectiva'}
Tópico descriptivo del grupo: retrospectiva
Tópico generalizador del grupo: retrospectiva

Grupo 3 : {'Opciones de capacitación interna'}
Tópico descriptivo del grupo: capacitación
Tópico generalizador del grupo: Opciones de capacitación interna

Grupo 4 : {'Falta de feedback'}
Tópico descriptivo del grupo: feedback
Tópico generalizador del grupo: Falta de feedback


In [6]:
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

# Crear modelo de Sentence Transformers
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

def encontrar_concepto_medio(topicos, modelo):
    embeddings = [modelo.encode([topico])[0] for topico in topicos]
    cov_matrix = np.cov(np.array(embeddings).T)
    U, S, V = np.linalg.svd(cov_matrix)
    direccion_promedio = V[0]
    direccion_promedio = direccion_promedio / np.linalg.norm(direccion_promedio)
    concepto_medio = np.mean(embeddings, axis=0) + direccion_promedio
    return concepto_medio

def encontrar_tópico_descriptivo(vector, palabras, modelo):
    vector = torch.tensor(vector, dtype=torch.float32)
    embeddings_palabras = torch.tensor(modelo.encode([palabras]), dtype=torch.float32)
    vecino_cercano = util.semantic_search(vector.reshape(1, -1), embeddings_palabras, top_k=1)[0][0]
    tópico_descriptivo = palabras[vecino_cercano['corpus_id']]
    return tópico_descriptivo

def agrupar_topicos(topicos, umbral=0.5):
    grupos = []
    
    for topico in topicos:
        embeddings_1 = model.encode([topico])[0]
        agregado = False
        for grupo in grupos:
            for t in grupo:
                embeddings_2 = model.encode([t])[0]
                similarity_score = cosine_similarity([embeddings_1], [embeddings_2])[0][0]
                if similarity_score > umbral:
                    grupo.add(topico)
                    agregado = True
                    break
            if agregado:
                break
        if not agregado:
            grupos.append({topico})
    
    return grupos

def encontrar_tópico_generalizador(topicos, modelo):
    embeddings = [modelo.encode([topico])[0] for topico in topicos]
    mean_embedding = np.mean(embeddings, axis=0)
    similitudes = [cosine_similarity([mean_embedding], [embedding])[0][0] for embedding in embeddings]
    index_tópico_generalizador = np.argmax(similitudes)
    tópico_generalizador = list(topicos)[index_tópico_generalizador]
    return tópico_generalizador

# Lista de tópicos
topicos = [
    'Análisis del capacity actual y propuestas a clientes',
    'retrospectiva',
    'Opciones de capacitación interna',
    'Seguimiento en el capacity y capacitación a freelancers',
    'Falta de feedback'
]

topicos = [
    'Adaptación al equipo y buen compañerismo',
    'Falta de tiempo para testear implementaciones',
    'Presión de los líderes del equipo',
    'Comidas poco llamativas en la oficina',
    'Espacios de trabajo no confortables',
    'Los espacios de trabajo no son suficientemente confortables',
    'Traumas por corregir errores y falta de recursos para pagar psicólogo'
]

# Lista de palabras relevantes para describir áreas de un equipo de trabajo
palabras_relevantes = [
    'comunicación',
    'colaboración',
    'eficiencia',
    'productividad',
    'coordinación',
    'creatividad',
    'compromiso'
]

# Calcular el concepto medio entre los tópicos
concepto_medio = encontrar_concepto_medio(topicos, model)

# Agrupar los tópicos
grupos = agrupar_topicos(topicos)

# Encontrar el tópico generalizador de cada grupo
for i, grupo in enumerate(grupos):
    print("\nGrupo", i+1, ":", grupo)
    
    # Encontrar el tópico descriptivo del grupo
    tópico_descriptivo = encontrar_tópico_descriptivo(concepto_medio, palabras_relevantes, model)
    print("Tópico descriptivo del grupo:", tópico_descriptivo)



Grupo 1 : {'Presión de los líderes del equipo', 'Adaptación al equipo y buen compañerismo'}
Tópico descriptivo del grupo: comunicación

Grupo 2 : {'Falta de tiempo para testear implementaciones'}
Tópico descriptivo del grupo: comunicación

Grupo 3 : {'Comidas poco llamativas en la oficina'}
Tópico descriptivo del grupo: comunicación

Grupo 4 : {'Los espacios de trabajo no son suficientemente confortables', 'Espacios de trabajo no confortables'}
Tópico descriptivo del grupo: comunicación

Grupo 5 : {'Traumas por corregir errores y falta de recursos para pagar psicólogo'}
Tópico descriptivo del grupo: comunicación


In [76]:
def encontrar_tópico_cercano(tópico, palabras_relevantes, modelo):
    embeddings_tópico = modelo.encode([tópico])[0]
    distancias = []
    for palabra in palabras_relevantes:
        embedding_palabra = modelo.encode([palabra])[0]
        distancia = cosine_similarity([embeddings_tópico], [embedding_palabra])[0][0]
        distancias.append(distancia)
    índice_tópico_cercano = np.argmax(distancias)
    return palabras_relevantes[índice_tópico_cercano], distancias[índice_tópico_cercano]

def agrupar_topicos(topicos, palabras_relevantes, umbral=0.1):
    grupos = {}
    
    for topico in topicos:
        tópico_cercano, distancia = encontrar_tópico_cercano(topico, palabras_relevantes, model)
        if distancia >= umbral:
            grupos.setdefault(tópico_cercano, []).append(topico)
        else:
            grupos.setdefault("Otros", []).append(topico)
    
    return grupos

# Lista de tópicos
topicos = [
    'Análisis del capacity actual y propuestas a clientes',
    'retrospectiva',
    'Opciones de capacitación interna',
    'Seguimiento en el capacity y capacitación a freelancers',
    'Falta de feedback'
]


topicos = [
    'Adaptación al equipo y buen compañerismo',
    'Falta de tiempo para testear implementaciones',
    'Presión de los líderes del equipo',
    'Comidas poco llamativas en la oficina',
    'Espacios de trabajo no confortables',
    'Los espacios de trabajo no son suficientemente confortables',
    'Traumas por corregir errores y falta de recursos para pagar psicólogo'
]


# Lista de palabras relevantes para describir áreas de un equipo de trabajo
palabras_relevantes = [
    'comunicación',
    'colaboración',
    'eficiencia',
    'productividad',
    'coordinación',
    'creatividad',
    'compromiso',
    'presion',
    'estres',
    'comodidad'
]

palabras_relevantes = [
    'cooperación',
    'comunicación',
    'eficiencia',
    'coordinación',
    'creatividad',
    'comodidad',
    'malestar',
    'capacitación',
    'Tiempo y velocidad',
    'Presión jerárquica'
    
    
]
# Agrupar los tópicos
grupos = agrupar_topicos(topicos, palabras_relevantes)

# Mostrar los grupos resultantes
for grupo, topicos_grupo in grupos.items():
    print("\nGrupo:", grupo)
    print("Tópicos:", topicos_grupo)



Grupo: capacitación
Tópicos: ['Análisis del capacity actual y propuestas a clientes', 'Opciones de capacitación interna', 'Seguimiento en el capacity y capacitación a freelancers']

Grupo: comunicación
Tópicos: ['retrospectiva', 'Falta de feedback']


In [5]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

# Crear modelo de Sentence Transformers
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')


def encontrar_tópico_cercano(tópico, palabras_relevantes, modelo):
    embeddings_tópico = modelo.encode([tópico])[0]
    distancias = []
    for palabra in palabras_relevantes:
        embedding_palabra = modelo.encode([palabra])[0]
        distancia = cosine_similarity([embeddings_tópico], [embedding_palabra])[0][0]
        distancias.append(distancia)
    índice_tópico_cercano = np.argmax(distancias)
    return palabras_relevantes[índice_tópico_cercano], distancias[índice_tópico_cercano]

def agrupar_topicos(topicos, palabras_relevantes, umbral=0.1):
    grupos = {}
    
    for topico in topicos:
        tópico_cercano, distancia = encontrar_tópico_cercano(topico, palabras_relevantes, model)
        if distancia >= umbral:
            grupos.setdefault(tópico_cercano, []).append(topico)
        else:
            grupos.setdefault("Otros", []).append(topico)
    
    return grupos


# Lista de tópicos
topicos = [
    'Análisis del capacity actual y propuestas a clientes',
    'retrospectiva',
    'Opciones de capacitación interna',
    'Seguimiento en el capacity y capacitación a freelancers',
    'Falta de feedback'
]


topicos = [
    'Adaptación al equipo y buen compañerismo',
    'Falta de tiempo para testear implementaciones',
    'Presión de los líderes del equipo',
    'Comidas poco llamativas en la oficina',
    'Espacios de trabajo no confortables',
    'Los espacios de trabajo no son suficientemente confortables',
    'Traumas por corregir errores y falta de recursos para pagar psicólogo'
]


# Lista de palabras relevantes para describir áreas de un equipo de trabajo
palabras_relevantes = [
    'comunicación',
    'colaboración',
    'eficiencia',
    'productividad',
    'coordinación',
    'creatividad',
    'compromiso',
    'presion',
    'estres',
    'comodidad'
]

palabras_relevantes = [
    'cooperación',
    'comunicación',
    'eficiencia',
    'coordinación',
    'creatividad',
    'comodidad',
    'malestar',
    'capacitación',
    'Tiempo y velocidad',
    'Presión jerárquica'
    
    
]
# Agrupar los tópicos
grupos = agrupar_topicos(topicos, palabras_relevantes)

    
print(grupos)


{'cooperación': ['Adaptación al equipo y buen compañerismo'], 'Tiempo y velocidad': ['Falta de tiempo para testear implementaciones'], 'Presión jerárquica': ['Presión de los líderes del equipo', 'Traumas por corregir errores y falta de recursos para pagar psicólogo'], 'comodidad': ['Comidas poco llamativas en la oficina', 'Espacios de trabajo no confortables', 'Los espacios de trabajo no son suficientemente confortables']}


In [8]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

# Crear modelo de Sentence Transformers
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')


def encontrar_tópico_cercano(tópico, palabras_relevantes, modelo):
    embeddings_tópico = modelo.encode([tópico])[0]
    embeddings_palabras_relevantes = modelo.encode(palabras_relevantes)
    
    # Calcular la similitud coseno entre el embedding del tópico y el conjunto de embeddings de palabras relevantes
    similitudes = cosine_similarity([embeddings_tópico], embeddings_palabras_relevantes)[0]
    
    # Encontrar el índice de la palabra más cercana
    índice_tópico_cercano = np.argmax(similitudes)
    
    # Devolver la palabra más cercana y su similitud coseno correspondiente
    return palabras_relevantes[índice_tópico_cercano], similitudes[índice_tópico_cercano]

def agrupar_topicos(topicos, palabras_relevantes, umbral=0.1):
    grupos = {}
    
    for topico in topicos:
        tópico_cercano, distancia = encontrar_tópico_cercano(topico, palabras_relevantes, model)
        if distancia >= umbral:
            grupos.setdefault(tópico_cercano, []).append(topico)
        else:
            grupos.setdefault("Otros", []).append(topico)
    
    return grupos

# Lista de tópicos
topicos = [
    'Análisis del capacity actual y propuestas a clientes',
    'retrospectiva',
    'Opciones de capacitación interna',
    'Seguimiento en el capacity y capacitación a freelancers',
    'Falta de feedback'
]



# Lista de palabras relevantes para describir áreas de un equipo de trabajo

palabras_relevantes = [
    'cooperación',
    'comunicación',
    'eficiencia',
    'coordinación',
    'creatividad',
    'comodidad',
    'malestar',
    'capacitación',
    'Tiempo y velocidad',
    'Presión jerárquica'
    
    
]

topicos = [
    'Adaptación al equipo y buen compañerismo',
    'Falta de tiempo para testear implementaciones',
    'Presión de los líderes del equipo',
    'Comidas poco llamativas en la oficina',
    'Espacios de trabajo no confortables',
    'Los espacios de trabajo no son suficientemente confortables',
    'Traumas por corregir errores y falta de recursos para pagar psicólogo'
]


# Agrupar los tópicos
grupos = agrupar_topicos(topicos, palabras_relevantes)


print(grupos)

{'cooperación': ['Adaptación al equipo y buen compañerismo'], 'Tiempo y velocidad': ['Falta de tiempo para testear implementaciones'], 'Presión jerárquica': ['Presión de los líderes del equipo', 'Traumas por corregir errores y falta de recursos para pagar psicólogo'], 'comodidad': ['Comidas poco llamativas en la oficina', 'Espacios de trabajo no confortables', 'Los espacios de trabajo no son suficientemente confortables']}


In [24]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

# Crear modelo de Sentence Transformers
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')


def encontrar_tópico_cercano(tópico, palabras_relevantes, modelo):
    embeddings_tópico = modelo.encode([tópico])[0]
    distancias = []
    for palabra in palabras_relevantes:
        embedding_palabra = modelo.encode([palabra])[0]
        distancia = cosine_similarity([embeddings_tópico], [embedding_palabra])[0][0]
        distancias.append(distancia)
    índice_tópico_cercano = np.argmax(distancias)
    return palabras_relevantes[índice_tópico_cercano], distancias[índice_tópico_cercano]

def agrupar_topicos(topicos, palabras_relevantes, umbral=0.25):
    grupos = {}
    
    for topico in topicos:
        tópico_cercano, distancia = encontrar_tópico_cercano(topico, palabras_relevantes, model)
        if distancia >= umbral:
            grupos.setdefault(tópico_cercano, []).append(topico)
        else:
            grupos.setdefault("Otros", []).append(topico)
    
    return grupos

a = [i for i in db['Test1'].find({}, {'_id': 0, 'team_id': 1, 'sprint': 1})]


for h,i in enumerate(a):

    
    if h > 8:
        print('------------------------------------------------------------------------------------------------------------------')
        print('------------------------------------------------------------------------------------------------------------------')
            
        team_id = i['team_id']
        sprint = i['sprint']
        
        topicos = list(db['Test1'].aggregate([
        {'$match': {'team_id': team_id, 'sprint': sprint}},
        {'$unwind': '$days'},
        {'$unwind': '$days.post'},
        {'$project': {
            '_id': 0,
            'topic': '$days.post.comment.topic'
        }},
        {'$match': {'topic': {'$exists': True, '$nin': ['', 'Sin tema relevante']}}}
    ]))
        
        topicos = [i['topic'] for i in topicos]


        palabras_relevantes = [
            'cooperación',
            'comunicación y trato',
            'productividad',
            'creatividad',
            'Entorno',
            'malestar y descontento',
            'Tiempo y velocidad',
            'Presión jerárquica',
            'capacitación'
            
            
        ]



        # Agrupar los tópicos
        grupos = agrupar_topicos(topicos, palabras_relevantes)

        # Mostrar los grupos resultantes
        for grupo, topicos_grupo in grupos.items():

            print("\nGrupo:", grupo)
            print("Tópicos:", topicos_grupo)

------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------

In [26]:
def encontrar_tópico_cercano(tópico, palabras_relevantes, modelo):
    embeddings_tópico = modelo.encode([tópico])[0]
    embeddings_palabras_relevantes = modelo.encode(palabras_relevantes)
    
    # Calcular la similitud coseno entre el embedding del tópico y el conjunto de embeddings de palabras relevantes
    similitudes = cosine_similarity([embeddings_tópico], embeddings_palabras_relevantes)[0]
    
    # Encontrar el índice de la palabra más cercana
    índice_tópico_cercano = np.argmax(similitudes)
    
    # Devolver la palabra más cercana y su similitud coseno correspondiente
    return palabras_relevantes[índice_tópico_cercano], similitudes[índice_tópico_cercano]

def agrupar_topicos(topicos, palabras_relevantes, umbral=0.28):
    grupos = {}
    
    for topico in topicos:
        tópico_cercano, distancia = encontrar_tópico_cercano(topico, palabras_relevantes, model)
        if distancia >= umbral:
            grupos.setdefault(tópico_cercano, []).append(topico)
        else:
            grupos.setdefault("Otros", []).append(topico)
    
    return grupos


a = [i for i in db['Test1'].find({}, {'_id': 0, 'team_id': 1, 'sprint': 1})]


for h,i in enumerate(a):

    
    if h > 8:
        print('------------------------------------------------------------------------------------------------------------------')
        print('------------------------------------------------------------------------------------------------------------------')
            
        team_id = i['team_id']
        sprint = i['sprint']
        
        topicos = list(db['Test1'].aggregate([
        {'$match': {'team_id': team_id, 'sprint': sprint}},
        {'$unwind': '$days'},
        {'$unwind': '$days.post'},
        {'$project': {
            '_id': 0,
            'topic': '$days.post.comment.topic'
        }},
        {'$match': {'topic': {'$exists': True, '$nin': ['', 'Sin tema relevante']}}}
    ]))
        
        topicos = [i['topic'] for i in topicos]


        palabras_relevantes = [
            'cooperación',
            'comunicación y trato',
            'productividad',
            'creatividad',
            'Entorno',
            'malestar y descontento',
            'Presión jerárquica',
            'capacitación'
            
            
        ]



        # Agrupar los tópicos
        grupos = agrupar_topicos(topicos, palabras_relevantes)

        # Mostrar los grupos resultantes
        for grupo, topicos_grupo in grupos.items():

            print("\nGrupo:", grupo)
            print("Tópicos:", topicos_grupo)

------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------

In [ ]:
team_id = '65c0eff74dc9f94d6eec80e4'

#team_id = '65bbb0e33d7928c51fe9ed9b'

team_id = '65afc105682fd36596103052'

topics = list(db['Test1'].aggregate([
    {'$match': {'team_id': team_id, 'sprint': sprint}},
    {'$unwind': '$days'},
    {'$unwind': '$days.post'},
    {'$project': {
        '_id': 0,
        'topic': '$days.post.comment.topic'
    }},
    {'$match': {'topic': {'$exists': True, '$nin': ['', 'Sin tema relevante']}}}
]))


topics = [i['topic'] for i in topics]

topics

In [74]:



def encontrar_tópico_cercano(tópico, palabras_relevantes, modelo):
    embeddings_tópico = modelo.encode([tópico])[0]
    embeddings_palabras_relevantes = modelo.encode(palabras_relevantes)
    
    # Calcular la similitud coseno entre el embedding del tópico y el conjunto de embeddings de palabras relevantes
    similitudes = cosine_similarity([embeddings_tópico], embeddings_palabras_relevantes)[0]
    
    # Encontrar el índice de la palabra más cercana
    índice_tópico_cercano = np.argmax(similitudes)
    
    # Devolver la palabra más cercana y su similitud coseno correspondiente
    return palabras_relevantes[índice_tópico_cercano], similitudes[índice_tópico_cercano]

def agrupar_topicos(topicos, palabras_relevantes, umbral=0.1):
    grupos = {}
    
    for topico in topicos:
        tópico_cercano, distancia = encontrar_tópico_cercano(topico, palabras_relevantes, model)
        if distancia >= umbral:
            grupos.setdefault(tópico_cercano, []).append(topico)
        else:
            grupos.setdefault("Otros", []).append(topico)
    
    return grupos

# Lista de tópicos
topicos = [
    'Análisis del capacity actual y propuestas a clientes',
    'retrospectiva',
    'Opciones de capacitación interna',
    'Seguimiento en el capacity y capacitación a freelancers',
    'Falta de feedback'
]




# Lista de palabras relevantes para describir áreas de un equipo de trabajo
palabras_relevantes = [
    'comunicación',
    'colaboración',
    'eficiencia',
    'productividad',
    'coordinación',
    'creatividad',
    'compromiso'
]

palabras_relevantes = [
    'cooperación',
    'comunicación',
    'eficiencia',
    'coordinación',
    'creatividad',
    'comodidad',
    'malestar',
    'capacitación',
    'Tiempo y velocidad',
    'Presión jerárquica'
    
    
]

topicos = [
    'Adaptación al equipo y buen compañerismo',
    'Falta de tiempo para testear implementaciones',
    'Presión de los líderes del equipo',
    'Comidas poco llamativas en la oficina',
    'Espacios de trabajo no confortables',
    'Los espacios de trabajo no son suficientemente confortables',
    'Traumas por corregir errores y falta de recursos para pagar psicólogo'
]


# Agrupar los tópicos
grupos = agrupar_topicos(topicos, palabras_relevantes)

# Mostrar los grupos resultantes
for grupo, topicos_grupo in grupos.items():
    print("\nGrupo:", grupo)
    print("Tópicos:", topicos_grupo)


Grupo: cooperación
Tópicos: ['Adaptación al equipo y buen compañerismo']

Grupo: Tiempo y velocidad
Tópicos: ['Falta de tiempo para testear implementaciones']

Grupo: Presión jerárquica
Tópicos: ['Presión de los líderes del equipo', 'Traumas por corregir errores y falta de recursos para pagar psicólogo']

Grupo: comodidad
Tópicos: ['Comidas poco llamativas en la oficina', 'Espacios de trabajo no confortables', 'Los espacios de trabajo no son suficientemente confortables']


test

In [13]:
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

# Crear modelo de Sentence Transformers
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

def encontrar_tópico_cercano(tópico, palabras_relevantes, modelo):
    embeddings_tópico = modelo.encode([tópico])[0]
    embeddings_palabras_relevantes = modelo.encode(palabras_relevantes)
    
    # Calcular la similitud coseno entre el embedding del tópico y el conjunto de embeddings de palabras relevantes
    similitudes = cosine_similarity([embeddings_tópico], embeddings_palabras_relevantes)[0]
    
    # Encontrar el índice de la palabra más cercana
    índice_tópico_cercano = np.argmax(similitudes)
    
    # Devolver la palabra más cercana y su similitud coseno correspondiente
    return palabras_relevantes[índice_tópico_cercano], similitudes[índice_tópico_cercano]

def agrupar_topicos(topicos, palabras_relevantes, umbral=0.1):
    grupos = {}
    
    for topico in topicos:
        tópico_cercano, distancia = encontrar_tópico_cercano(topico, palabras_relevantes, model)
        if distancia >= umbral:
            grupos.setdefault(tópico_cercano, []).append(topico)
        else:
            grupos.setdefault("Otros", []).append(topico)
            
    return grupos


a = [i for i in db['Test1'].find({}, {'_id': 0, 'team_id': 1, 'sprint': 1})]


for h,i in enumerate(a):

    
    if h > 8:
        print('------------------------------------------------------------------------------------------------------------------')
        print('------------------------------------------------------------------------------------------------------------------')
            
        team_id = i['team_id']
        sprint = i['sprint']
        
        topicos = list(db['Test1'].aggregate([
        {'$match': {'team_id': team_id, 'sprint': sprint}},
        {'$unwind': '$days'},
        {'$unwind': '$days.post'},
        {'$project': {
            '_id': 0,
            'topic': '$days.post.comment.topic'
        }},
        {'$match': {'topic': {'$exists': True, '$nin': ['', 'Sin tema relevante']}}}
    ]))
        
        topicos = [i['topic'] for i in topicos]


        palabras_relevantes = [
            'cooperación',
            'comunicación',
            'eficiencia',
            'coordinación',
            'creatividad',
            'comodidad',
            'malestar y descontento',
            'capacitación',
            'Tiempo y velocidad',
            'Presión jerárquica'
            
            
        ]



        # Agrupar los tópicos
        grupos = agrupar_topicos(topicos, palabras_relevantes)

        # Mostrar los grupos resultantes
        for grupo, topicos_grupo in grupos.items():

            print("\nGrupo:", grupo)
            print("Tópicos:", topicos_grupo)

------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------

In [8]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

# Crear modelo de Sentence Transformers
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

def encontrar_tópico_cercano(tópico, palabras_relevantes, modelo):
    embeddings_tópico = modelo.encode([tópico])[0]
    distancias = []
    for palabra in palabras_relevantes:
        embedding_palabra = modelo.encode([palabra])[0]
        distancia = cosine_similarity([embeddings_tópico], [embedding_palabra])[0][0]
        distancias.append(distancia)
    índice_tópico_cercano = np.argmax(distancias)
    return palabras_relevantes[índice_tópico_cercano], distancias[índice_tópico_cercano]

def agrupar_topicos(topicos, palabras_relevantes, umbral=0.25):
    grupos = {}
    
    for topico in topicos:
        tópico_cercano, distancia = encontrar_tópico_cercano(topico, palabras_relevantes, model)
        if distancia >= umbral:
            grupos.setdefault(tópico_cercano, []).append(topico)
        else:
            grupos.setdefault("Otros", []).append(topico)
    
    return grupos


a = [i for i in db['Test1'].find({}, {'_id': 0, 'team_id': 1, 'sprint': 1})]


for h,i in enumerate(a):

    
    if h > 8:
        print('------------------------------------------------------------------------------------------------------------------')
        print('------------------------------------------------------------------------------------------------------------------')
            
        team_id = i['team_id']
        sprint = i['sprint']
        
        topicos = list(db['Test1'].aggregate([
        {'$match': {'team_id': team_id, 'sprint': sprint}},
        {'$unwind': '$days'},
        {'$unwind': '$days.post'},
        {'$project': {
            '_id': 0,
            'topic': '$days.post.comment.topic'
        }},
        {'$match': {'topic': {'$exists': True, '$nin': ['', 'Sin tema relevante']}}}
    ]))
        
        topicos = [i['topic'] for i in topicos]


        palabras_relevantes = [
            'cooperación',
            'comunicación y trato',
            'productividad',
            'creatividad',
            'Entorno',
            'malestar y descontento',
            'Tiempo y velocidad',
            'Presión jerárquica'
            
            
        ]



        # Agrupar los tópicos
        grupos = agrupar_topicos(topicos, palabras_relevantes)

        # Mostrar los grupos resultantes
        for grupo, topicos_grupo in grupos.items():

            print("\nGrupo:", grupo)
            print("Tópicos:", topicos_grupo)

------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity

# Crear modelo de Sentence Transformers
model = SentenceTransformer('hiiamsid/sentence_similarity_spanish_es')

def encontrar_tópico_cercano(tópico, palabras_relevantes, modelo):
    embeddings_tópico = modelo.encode([tópico])[0]
    distancias = []
    for palabra in palabras_relevantes:
        embedding_palabra = modelo.encode([palabra])[0]
        distancia = cosine_similarity([embeddings_tópico], [embedding_palabra])[0][0]
        distancias.append(distancia)
    índice_tópico_cercano = np.argmax(distancias)
    return palabras_relevantes[índice_tópico_cercano], distancias[índice_tópico_cercano]

def agrupar_topicos(topicos, palabras_relevantes, umbral=0.25):
    grupos = {}
    
    for topico in topicos:
        tópico_cercano, distancia = encontrar_tópico_cercano(topico, palabras_relevantes, model)
        if distancia >= umbral:
            grupos.setdefault(tópico_cercano, []).append(topico)
        else:
            grupos.setdefault("Otros", []).append(topico)
    
    return grupos